In [ ]:
!pip install matplotlib==3.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.2 MB 4.2 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 960 kB 43.4 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


In [ ]:
import numpy as np
from numpy import *
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
plt.rcParams['figure.figsize'] = [6.0, 6.0]
class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs
    
    def do_3d_projection(self, renderer=None):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, self.axes.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))

        return np.min(zs)

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]),)
        FancyArrowPatch.draw(self, renderer)

import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

    



In [ ]:
# METHODS
beta = 0.4
h = 1e-2
alpha = 1e-4
decay_steps=  10
import torch
import math
import decimal
import copy
class Ours:
  def __init__(self, params, steps, h, loss_func,plr,beta=0.9, eps=1e-3, term_eps=1e-3):
    self.params = params
    self.radius = 0
    self.eps = eps
    self.p1 = None
    self.p2 = None
    self.p1_grad = None
    self.p2_grad = None
    self.steps = steps
    self.h = h
    self.loss_func = loss_func
    self.plr = plr
    self.iter = 0
    self.avg_theta = 0
    self.h_backup = h
    self.p_moment = 0
    self.beta = beta
    self.avg_depth = 0
    self.term_eps = 0
    self.iter = 0
    self.flag = 1
    self.avg_sum=0
    self.avg_sum = 0
    self.avg_radius = torch.tensor([0], device='cpu')
    self.plane_radius = torch.tensor([0], device='cpu')
    self.set_locality_flag = 1


  def make_rand_vector(self, dims):
      vec = torch.rand(dims, requires_grad=True)

      return vec

  def angle(self, a, b):
      a = a.detach()
      b = b.detach()
      a = a/torch.norm(a)
      b = b/torch.norm(b)

      term = torch.tensor([float(decimal.Decimal(torch.norm(a-b).item())/decimal.Decimal(torch.norm(a+b).item()))], device='cpu')

      angle = 2 * torch.atan(term)
      return angle  #* 180/torch.pi

  def projection(self,p):
    v = p - self.plane_point
    projected_normal = torch.dot(v, self.plane_normal)
    projected_point = p - projected_normal
    return projected_point


  def step(self, loss):
    self.iter += 1
    flag = 0

    self.p1 = copy.deepcopy(self.params)
    grad_x = torch.autograd.grad(loss, self.params, create_graph=True, retain_graph=True )
    grad_x_vec = torch.cat([g.contiguous().view(-1) for g in grad_x])#.reshape(-1,1)
    grad_x_vec_d = grad_x_vec.detach()
    norm_grad = torch.norm(grad_x_vec, 2)
    if norm_grad <= self.term_eps:
      return

    self.p1_grad = grad_x_vec
    self.p1_test = 0

    ## GOING PERPENDICULAR

    rand_vec = self.make_rand_vector(self.p1_grad.shape).to('cpu')
    perp_vec = rand_vec - ((rand_vec.T @ self.p1_grad)/(torch.pow(torch.norm(self.p1_grad,2),2))) * self.p1_grad
    self.rand_vec = rand_vec

    norm_len = torch.norm(perp_vec, 2)


    perp_vec = (perp_vec / (torch.norm(perp_vec, 2)))

    
    for (opt_param, p1_param) in zip(self.params, self.p1):
        opt_param.data.fill_(p1_param.data)
    


    step_length = 0



    self.perp_vec = perp_vec
    self.grad_normal = grad_x_vec



    step_length += torch.norm(perp_vec.detach(), 2) * self.h #/ (torch.sqrt(self.p_moment) + 1e-7)
    index = 0
    for p in self.params:
        term = perp_vec[index: index + p.numel()].detach().reshape(p.shape)

        p.data.add_((-1) * (term * self.h))
        index += p.numel()

    loss_val = self.loss_func()

    grad_x = torch.autograd.grad(loss_val, self.params, create_graph=True,retain_graph=True )
    grad_x_vec = torch.cat([g.contiguous().view(-1) for g in grad_x])#.reshape(-1,1)
    grad_x_vec_d = grad_x_vec.detach()
    self.p2_grad = grad_x_vec.clone()
    ang_g1g2 = math.ceil(self.angle(self.p1_grad, self.p2_grad)* 180/torch.pi)
    ang_p1g2 = math.ceil(self.angle(self.perp_vec, self.p2_grad) * 180/torch.pi)

    if(ang_p1g2 < 90):
        self.p_moment = 0
        #self.avg_locality =0
        #self.iter = 0
        self.flag = 1

    #if(ang_g1g2 <= 10 and ang_p1g2 > 90 ): #
    #    #print("OKAY")
    #    self.h = self.h_backup
    #    flag = 1
    #else:
        #print("G1_G2", ang_g1g2, "P1 G2", ang_p1g2, " H ", self.h, "DISTANCE ", torch.norm(self.p1_grad - self.p2_grad, 2).item())
    #    self.h = self.h / 10


    #    #print(torch.norm(p3_vec))
    #    #print(" TRAPPED ", torch.norm(param_vec))
    #    flag = 0

    # DONE PERPENDICULAR

    #print(self.p1)
    self.p2 = copy.deepcopy(self.params)
    p1_vec = torch.cat([g.contiguous().view(-1) for g in self.p1])

    self.theta = self.angle(self.p1_grad, self.p2_grad) + self.eps
    #print("THETA " , self.theta.item() * 180/torch.pi)
    import time
    #time.sleep(0.5)

    #step_length = self.h * self.steps
    self.radius = step_length * (torch.cos(self.theta)/torch.sin(self.theta))

    index = 0
    #self.params = copy.deepcopy(self.p1)
    for param,p1_param in zip(self.params, self.p1):
        param.data.fill_(p1_param.data)

    if(self.radius < self.avg_radius):
        #print("\nJUMPING ",self.flag, "\n")
        self.radius = 2.0 * self.radius
    self.normalized_grad = (self.p1_grad/(torch.norm(self.p1_grad,2))) * self.radius

    for p in self.params:

        term = self.normalized_grad[index: index + p.numel()].detach().reshape(p.shape)
        #print("TERM ", term)
        p.data.add_((-1)*term)
        index += p.numel()
    p2_vec = torch.cat([g.contiguous().view(-1) for g in self.p2])
    #print(" DISTANCE     ", torch.norm(p1_vec-p2_vec))
    self.avg_theta += self.theta * 180/torch.pi
    self.avg_theta = self.avg_theta/self.iter
    self.avg_radius = beta * self.avg_radius + (1-beta) * self.radius
    


In [ ]:
class AngularGradCos(object):
    def __init__(self, params,lrn_rate, beta1, beta2, eps):
        self.x = params
        self.params = params
        self.lrn_rate = lrn_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.idx = 0
        self.m = None # 1st order
        self.v = None # 2nd order
        self.g_prev = 0.0
        self.min = 0.0

    def step(self, loss_val):
        self.idx += 1
        g = torch.autograd.grad(loss_val, self.params, create_graph=True,retain_graph=True )[0].detach()
        self.g = g
        if self.m == None or self.v == None:
          self.m = torch.zeros_like(g)
          self.v = torch.zeros_like(g)
        self.m = self.beta1 * self.m + (1.0 - self.beta1) * g
        self.v = self.beta2 * self.v + (1.0 - self.beta2) * g ** 2
        m_adj = self.m / (1.0 - np.power(self.beta1, self.idx))
        v_adj = self.v / (1.0 - np.power(self.beta2, self.idx))

        
        tan_theta = abs((self.g_prev - g) / (1 + self.g_prev * g))
        cos_theta = 1 / torch.sqrt(1 + torch.square(tan_theta))
        angle = np.arctan(tan_theta) * (180 / 3.141592653589793238)
      
        
        self.min = angle
        diff = abs(self.g_prev - g)
        final_cos_theta = cos_theta


        dfc = 1.0 / (1.0 + np.exp(final_cos_theta))
        term_ex =  self.lrn_rate * m_adj * dfc / (np.sqrt(v_adj) + self.eps)
        index = 0
        for p in self.params:

          term = term_ex[index: index + p.numel()].detach().reshape(p.shape)
          #print("TERM ", term)
          p.data.add_((-1)*term)
          self.g_prev = g
          index += p.numel()
        
class Adam(object):
    def __init__(self, params,  lrn_rate, beta1, beta2, eps):
        self.lrn_rate = lrn_rate
        self.params = params
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.idx = 0
        self.m = None # 1st order
        self.v = None # 2nd order

    def step(self, loss_val):
        self.idx += 1
        g = torch.autograd.grad(loss_val, self.params, create_graph=True,retain_graph=True )[0].detach()
        self.g = g
        if self.m == None or self.v == None:
          self.m = torch.zeros_like(g)
          self.v = torch.zeros_like(g)
        self.m = self.beta1 * self.m + (1.0 - self.beta1) * g
        self.v = self.beta2 * self.v + (1.0 - self.beta2) * g ** 2
        m_adj = self.m / (1.0 - np.power(self.beta1, self.idx))
        v_adj = self.v / (1.0 - np.power(self.beta2, self.idx))
        term_ex = self.lrn_rate * m_adj / np.sqrt(v_adj + self.eps)
        index = 0
        for p in self.params:

          term = term_ex[index: index + p.numel()].detach().reshape(p.shape)
          #print("TERM ", term)
          p.data.add_((-1)*term)
          self.g_prev = g
          index += p.numel()

class RMSprop(object):
    def __init__(self, params,  lrn_rate, beta1, beta2, eps):
        self.lrn_rate = lrn_rate
        self.params = params
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.idx = 0
        self.m = None # 1st order
        self.v = None # 2nd order

    def step(self, loss_val):
        self.idx += 1
        g = torch.autograd.grad(loss_val, self.params, create_graph=True,retain_graph=True )[0].detach()
        self.g = g
        if self.m == None or self.v == None:
          self.m = torch.zeros_like(g)
          self.v = torch.zeros_like(g)
        self.m = self.beta1 * self.m + (1.0 - self.beta1) * g
        self.v = self.beta2 * self.v + (1.0 - self.beta2) * g ** 2
        m_adj = self.m / (1.0 - np.power(self.beta1, self.idx))
        v_adj = self.v / (1.0 - np.power(self.beta2, self.idx))
        term_ex = self.lrn_rate * g / np.sqrt(v_adj + self.eps)
        index = 0
        for p in self.params:

          term = term_ex[index: index + p.numel()].detach().reshape(p.shape)
          #print("TERM ", term)
          p.data.add_((-1)*term)
          self.g_prev = g
          index += p.numel()

class AdaBelief(object):
    def __init__(self, params,  lrn_rate, beta1, beta2, eps):
        self.lrn_rate = lrn_rate
        self.params = params
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.idx = 0
        self.m = None # 1st order
        self.v = None # 2nd order

    def step(self, loss_val):
        self.idx += 1
        g = torch.autograd.grad(loss_val, self.params, create_graph=True,retain_graph=True )[0].detach()
        self.g = g
        if self.m == None or self.v == None:
          self.m = torch.zeros_like(g)
          self.v = torch.zeros_like(g)
        self.m = self.beta1 * self.m + (1.0 - self.beta1) * g
        self.v = self.beta2 * self.v + (1.0 - self.beta2) * (g - self.m) ** 2 + self.eps
        m_adj = self.m / (1.0 - np.power(self.beta1, self.idx))
        v_adj = self.v / (1.0 - np.power(self.beta2, self.idx))
        term_ex = self.lrn_rate * m_adj / np.sqrt(v_adj + self.eps)
        index = 0
        for p in self.params:

          term = term_ex[index: index + p.numel()].detach().reshape(p.shape)
          #print("TERM ", term)
          p.data.add_((-1)*term)
          self.g_prev = g
          index += p.numel()

class diffGrad(object):
    def __init__(self, params, lrn_rate, beta1, beta2, eps):
        self.lrn_rate = lrn_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.params = params
        self.eps = eps
        self.idx = 0
        self.m = None # 1st order
        self.v = None # 2nd order
        self.g_prev = 0.0

    def step(self, loss_val):
        self.idx += 1
        g = torch.autograd.grad(loss_val, self.params, create_graph=True,retain_graph=True )[0].detach()
        self.g = g
        if self.m == None or self.v == None:
          self.m = torch.zeros_like(g)
          self.v = torch.zeros_like(g)
        self.m = self.beta1 * self.m + (1.0 - self.beta1) * g
        self.v = self.beta2 * self.v + (1.0 - self.beta2) * g ** 2
        m_adj = self.m / (1.0 - np.power(self.beta1, self.idx))
        v_adj = self.v / (1.0 - np.power(self.beta2, self.idx))
        dfc = 1.0 / (1.0 + np.exp(-np.abs(self.g_prev - g)))
        term_ex = self.lrn_rate * m_adj * dfc / (np.sqrt(v_adj) + self.eps)
        index = 0
        for p in self.params:

          term = term_ex[index: index + p.numel()].detach().reshape(p.shape)
          #print("TERM ", term)
          p.data.add_((-1)*term)
          self.g_prev = g
          index += p.numel()
        self.g_prev = g
        
class AngularGradTan(object):
    def __init__(self, params, lrn_rate, beta1, beta2, eps):
        self.lrn_rate = lrn_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.params = params
        self.eps = eps
        self.idx = 0
        self.m = None # 1st order
        self.v = None # 2nd order
        self.g_prev = 0.0
        self.min = 0.0

    def step(self, loss_val):
        self.idx += 1
        g = torch.autograd.grad(loss_val, self.params, create_graph=True,retain_graph=True )[0].detach()
        self.g = g
        if self.m == None or self.v == None:
          self.m = torch.zeros_like(g)
          self.v = torch.zeros_like(g)
        self.m = self.beta1 * self.m + (1.0 - self.beta1) * g
        self.v = self.beta2 * self.v + (1.0 - self.beta2) * g ** 2
        m_adj = self.m / (1.0 - np.power(self.beta1, self.idx))
        v_adj = self.v / (1.0 - np.power(self.beta2, self.idx))

        tan_theta = abs((self.g_prev - g) / (1 + self.g_prev * g))
        cos_theta = 1 / np.sqrt(1 + tan_theta**2)
        angle = np.arctan(tan_theta) * (180 / 3.141592653589793238)
     
        
        self.min = angle
        diff = abs(self.g_prev - g)
        final_tan_theta = tan_theta

        dfc = 1.0 / (1.0 + np.exp(final_tan_theta))
        term_ex = self.lrn_rate * m_adj * dfc / (np.sqrt(v_adj) + self.eps)
        index = 0
        for p in self.params:

          term = term_ex[index: index + p.numel()].detach().reshape(p.shape)
          #print("TERM ", term)
          p.data.add_((-1)*term)
          self.g_prev = g
          index += p.numel()
        self.g_prev = g
        


class SGDM(object):
    def __init__(self, params, lrn_rate, beta1, eps):
        self.lrn_rate = lrn_rate
        self.beta1 = beta1
        self.params = params
        self.eps = eps
        self.idx = 0
        self.m = None # 1st order
        self.v = None # 2nd order
        
    def step(self, loss_val):
        self.idx += 1
        g = torch.autograd.grad(loss_val, self.params, create_graph=True,retain_graph=True )[0].detach()
        self.g = g
        if self.m == None or self.v == None:
          self.m = torch.zeros_like(g)
          self.v = torch.zeros_like(g)
        self.m = self.beta1 * self.m + (1.0 - self.beta1) * g
        
        m_adj = self.m / (1.0 - np.power(self.beta1, self.idx))
        term_ex =  self.lrn_rate * m_adj
        index = 0
        for p in self.params:

          term = term_ex[index: index + p.numel()].detach().reshape(p.shape)
          #print("TERM ", term)
          p.data.add_((-1)*term)
          self.g_prev = g
          index += p.numel()

# Function definition

In [ ]:
import torch
def func_1():
  global var
  x = var
  x1 = x[0]
  x2 = x[0]

  return -torch.sin(x1) * torch.pow(x2,2)

def func_1_graph(x):
  x1 = x[0]
  x2 = x[0]

  return -np.sin(x1) * pow(x2,2)

def func_2_graph(x):
  x1 = x[0]
  x2 = x[1]
  return -np.sin((pow(x1,2) + pow(x2,2)))/( pow(x1,2) + pow(x2,2))


def func_2():
  global var
  x = var
  x1 = x[0]
  x2 = x[1]

  return -torch.sin((pow(x1,2) + pow(x2,2)))/( pow(x1,2) + pow(x2,2))


def func_3_graph(x):

  x1 = x[0]
  x2 = x[1]

  q1 = torch.full(x1.shape, -0.65)
  q2 = torch.full(x1.shape, -0.35)
  q3 = torch.full(x1.shape, -0.05)
  q4 = torch.full(x1.shape, 0.25)
  q5 = torch.full(x1.shape, 0.55)

  t1 = torch.add(torch.sign(q1-x1), torch.sign(q2-x1))
  t2 = torch.add(t1, torch.sign(q3-x1))
  t3 = torch.add(t1, t2)
  t4 = torch.add(torch.sign(q4-x1), torch.sign(q5-x1))
  t5 = torch.add(t3,t4)
  


  return ((t5.numpy())/7+np.sin(5*x1)*np.cos(5*x2)/100)

def func_3():
  global var
  x = var
  x1 = x[0]
  x2 = x[1]

  q1 = torch.full_like(x1, -0.65)
  q2 = torch.full_like(x1, -0.35)
  q3 = torch.full_like(x1, -0.05)
  q4 = torch.full_like(x1, 0.25)
  q5 = torch.full_like(x1, 0.55)

  t1 = torch.add(torch.sign(q1-x1), torch.sign(q2-x1))
  t2 = torch.add(t1, torch.sign(q3-x1))
  t3 = torch.add(t1, t2)
  t4 = torch.add(torch.sign(q4-x1), torch.sign(q5-x1))
  t5 = torch.add(t3,t4)
  


  return ((t5)/7 + torch.sin(5*x1)*torch.cos(5*x2)/100)


#Function 1 train and plot

In [ ]:
# LOOP 


# TRAINING LOOP
epochs = 1000
func = func_1##func_3
func_graph = func_1_graph##func_3_graph

lrn_rate = 1e-2
beta1 = 0.95
beta2 = 0.999
eps = 1e-2
eps_t = 1e-5


for i in range(8):
  var = torch.tensor([-2.0,0.0], requires_grad=True)
  optimizers = {
    "Ours" :  Ours(params=var, steps=1, h=lrn_rate, loss_func= func,plr=h,beta=0.9,eps = eps,term_eps=eps_t),
    "AngularGradCos" :  AngularGradCos(params = var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps),
    "AngularGradTan" : AngularGradTan(params = var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps),
    "Adam" : Adam(params=var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps),
    "RMSprop" : RMSprop(params=var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps),
    "AdaBelief" : AdaBelief(params=var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps),
    "diffGrad" : diffGrad(params=var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps),
    "SGDM" : SGDM(params=var, lrn_rate=lrn_rate, beta1=beta1, eps=eps),
    

  }
  method_name = list(optimizers.keys())[i]
  optimizer = optimizers[method_name]
  print(method_name)
  points = []
  x_coord, y_coord = optimizer.params
  z_coord = func_graph(optimizer.params.detach())
  x_coord = x_coord.item()
  y_coord = y_coord.item()
  z_coord = z_coord.item()
  points.append([x_coord, y_coord, z_coord])
  
  for epoch in range(epochs):
    #print(epoch)
    loss_value = func()
    optimizer.step(loss_value)
    x_coord, y_coord = optimizer.params
    z_coord = func_graph(optimizer.params.detach())
    x_coord = x_coord.item()
    y_coord = y_coord.item()
    z_coord = z_coord.item()
    points.append([x_coord, y_coord, z_coord])

    
  

  points = np.array(points)
  #points = np.array([[-2,2,func_1([-2,2])], [-2,0, func_1([-2,0])],])# [-2,1, func_1([-2,1])], [0,0, func_1([0,0])]])
  fig = plt.figure()
  axes = fig.gca(projection ='3d', computed_zorder=False)
  #print("OPTIMAL ", optimizer.params)

  x = np.arange(-3,3,0.1)
  y = np.arange(-3,3,0.1)
  axis_x,axis_y = np.meshgrid(x,y)



  z = func_graph([axis_x, axis_y])
  #print(axis_x)

  surf = axes.plot_surface(axis_x, axis_y, z, alpha=0.6, zorder=0,)
  surf._facecolors2d = surf._facecolor3d
  surf._edgecolors2d = surf._edgecolor3d
  #axes.plot3D(points[:,0], points[:,1], points[:,2],'red')

  for i,v in enumerate(points[:len(points)-1]):
      #ax.plot([mean_x,v[0]], [mean_y,v[1]], [mean_z,v[2]], color='red', alpha=0.8, lw=3)
      #I will replace this line with:
      a = Arrow3D([points[i,0], points[i+1, 0]], [points[i,1], points[i+1, 1]], 
                  [points[i,2], points[i+1, 2]], mutation_scale=10, 
                  lw=2, arrowstyle="-|>", color="red")
      a.set_zorder(1)
      axes.add_artist(a)

  axes.set_xlabel("x")
  axes.set_ylabel("y")
  axes.set_zlabel("function value", rotation=45)
  axes.set_title(method_name,fontsize=20)

  for point in points[1:len(points)-1]:
    axes.scatter3D(points[:,0], points[:,1], points[:,2], c='red', zorder = 3)
  axes.scatter3D(-2.0,0.0, func_graph(torch.tensor([-2.0,0.0])), s = 140, c='orange' ,marker='*' , zorder = 3,label='start')
  axes.scatter3D(points[-1,0], points[-1,1], points[-1,2], s=140, c='black' ,marker='*' , zorder = 3, label= 'end')
  axes.legend(loc="upper right", bbox_to_anchor=(1.15,0.8))
  axes.view_init(10, 60)
  #axes.view_init(40, 60)
  fig.savefig(method_name+".png", dpi=300, format='png', bbox_inches='tight')
  fig.clf()
  plt.clf()

Ours


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: MatplotlibDeprecationWarning: 
The M attribute was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use self.axes.M instead.


AngularGradCos
AngularGradTan
Adam
RMSprop
AdaBelief
diffGrad
SGDM


<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

...

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

#Function 2 train and plot

In [ ]:
# LOOP 


# TRAINING LOOP
epochs = 1000
func = func_2##func_3
func_graph = func_2_graph##func_3_graph
#optimizer =  new_gd_triangle_dl_refined_depth_skipping(params=var, steps=1, h=h, loss_func= func,plr=h,beta=0.9)


lrn_rate = 1e-2
beta1 = 0.95
beta2 = 0.999
eps = 1e-3#0.00000001
eps_t = 1e-5
start_point = [3.0,3.0]
#optimizer = AngularGradCos(params = var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps)
#optimizer = AngularGradTan(params = var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps)
#optimizer = Adam(params=var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps)
#optimizer = AdaBelief(params=var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps)
#optimizer = diffGrad(params=var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps)
#optimizer = SGDM(params=var, lrn_rate=lrn_rate, beta1=beta1, eps=eps)


for i in range(8):
  var = torch.tensor(start_point, requires_grad=True)
  optimizers = {
    "Ours" :  Ours(params=var, steps=1, h=lrn_rate, loss_func= func,plr=h,beta=0.9, eps=eps,term_eps=eps_t),
    "Ours" :  Ours(params=var, steps=1, h=lrn_rate, loss_func= func,plr=h,beta=0.9, eps=eps,term_eps=eps_t),
    "AngularGradCos" :  AngularGradCos(params = var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps),
    "AngularGradTan" : AngularGradTan(params = var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps),
    "Adam" : Adam(params=var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps),
    "RMSprop" : RMSprop(params=var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps),
    "AdaBelief" : AdaBelief(params=var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps),
    "diffGrad" : diffGrad(params=var, lrn_rate=lrn_rate, beta1=beta1, beta2=beta2, eps=eps),
    "SGDM" : SGDM(params=var, lrn_rate=lrn_rate, beta1=beta1, eps=eps),
    "Ours" :  Ours(params=var, steps=1, h=lrn_rate, loss_func= func,plr=h,beta=0.9, eps=eps,term_eps=eps_t),
    
    

  }
  method_name = list(optimizers.keys())[i]
  optimizer = optimizers[method_name]
  print(method_name)
  points = []
  x_coord, y_coord = optimizer.params
  z_coord = func_graph(optimizer.params.detach())
  x_coord = x_coord.item()
  y_coord = y_coord.item()
  z_coord = z_coord.item()
  points.append([x_coord, y_coord, z_coord])
  
  for epoch in range(epochs):
    #print(epoch)
    loss_value = func()
    optimizer.step(loss_value)
    x_coord, y_coord = optimizer.params
    z_coord = func_graph(optimizer.params.detach())
    x_coord = x_coord.item()
    y_coord = y_coord.item()
    z_coord = z_coord.item()
    points.append([x_coord, y_coord, z_coord])

    
  

  points = np.array(points)
  #points = np.array([[-2,2,func_1([-2,2])], [-2,0, func_1([-2,0])],])# [-2,1, func_1([-2,1])], [0,0, func_1([0,0])]])
  fig = plt.figure()
  axes = fig.gca(projection ='3d', computed_zorder=False)
  #print("OPTIMAL ", optimizer.params)

  x = np.arange(-4,4,0.1)
  y = np.arange(-4,4,0.1)
  axis_x,axis_y = np.meshgrid(x,y)



  z = func_graph([axis_x, axis_y])
  #print(axis_x)

  surf = axes.plot_surface(axis_x, axis_y, z, alpha=0.6, zorder=0,)
  surf._facecolors2d = surf._facecolor3d
  surf._edgecolors2d = surf._edgecolor3d
  #axes.plot3D(points[:,0], points[:,1], points[:,2],'red')

  for i,v in enumerate(points[:len(points)-1]):
      #ax.plot([mean_x,v[0]], [mean_y,v[1]], [mean_z,v[2]], color='red', alpha=0.8, lw=3)
      #I will replace this line with:
      a = Arrow3D([points[i,0], points[i+1, 0]], [points[i,1], points[i+1, 1]], 
                  [points[i,2], points[i+1, 2]], mutation_scale=10, 
                  lw=2, arrowstyle="-|>", color="red")
      a.set_zorder(1)
      axes.add_artist(a)

  axes.set_xlabel("x")
  axes.set_ylabel("y")
  axes.set_zlabel("function value", rotation=45)
  axes.set_title(method_name,fontsize=20)

  for point in points[1:len(points)-1]:
    axes.scatter3D(points[:,0], points[:,1], points[:,2], c='red', zorder = 3)
  axes.scatter3D(start_point[0],start_point[1], func_graph(torch.tensor(start_point)), s = 140, c='orange' ,marker='*' , zorder = 3,label='start')
  axes.scatter3D(points[-1,0], points[-1,1], points[-1,2], s=140, c='black' ,marker='*' , zorder = 3, label= 'end')
  axes.legend(loc="upper right", bbox_to_anchor=(1.15,0.8))
  axes.view_init(40, 60)
  #axes.view_init(40, 60)
  fig.savefig(method_name+".png", dpi=100, format='png', bbox_inches='tight')
  fig.clf()
  plt.clf()

Ours


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: MatplotlibDeprecationWarning: 
The M attribute was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use self.axes.M instead.


AngularGradCos
AngularGradTan
Adam
RMSprop
AdaBelief
diffGrad
SGDM


<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

...

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>